In [148]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.test.utils import datapath

from anytree import Node, RenderTree
from pandas import Series

In [154]:
#with open('evaluation_data/R-HSA-1474290/gene_sets.json') as f:
#    data = json.load(f, object_hook=lambda elem: print(elem))
#wv_from_bin = KeyedVectors.load_word2vec_format("__data/nlp/PubMed-Pyysalo-2013/wikipedia-pubmed-and-PMC-w2v.bin", binary=True)  # C binary format
#wv_from_bin["scna"]


In [171]:
"__data/ppi/BioGrid/BIOGRID-ALL-3.5.166.mitab.txt"

,#ID Interactor A,ID Interactor B,Alt IDs Interactor A,Alt IDs Interactor B,Aliases Interactor A,Aliases Interactor B,Interaction Detection Method,Publication 1st Author,Publication Identifiers,Taxid Interactor A,Taxid Interactor B,Interaction Types,Source Database,Interaction Identifiers,Confidence Values,FromId,ToId,FromTaxID,ToTaxID
0,entrez gene/locuslink:6416,entrez gene/locuslink:2318,biogrid:112315|entrez gene/locuslink:MAP2K4,biogrid:108607|entrez gene/locuslink:FLNC,entrez gene/locuslink:JNKK(gene name synonym)|...,entrez gene/locuslink:ABP-280(gene name synony...,"psi-mi:""MI:0018""(two hybrid)",Marti A (1997),pubmed:9006895,taxid:9606,taxid:9606,"psi-mi:""MI:0407""(direct interaction)","psi-mi:""MI:0463""(biogrid)",biogrid:103,-,6416,2318,9606,9606
1,entrez gene/locuslink:84665,entrez gene/locuslink:88,biogrid:124185|entrez gene/locuslink:MYPN,biogrid:106603|entrez gene/locuslink:ACTN2,entrez gene/locuslink:CMD1DD(gene name synonym...,entrez gene/locuslink:CMD1AA(gene name synonym),"psi-mi:""MI:0018""(two hybrid)",Bang ML (2001),pubmed:11309420,taxid:9606,taxid:9606,"psi-mi:""MI:0407""(direct interaction)","psi-mi:""MI:0463""(biogrid)",biogrid:117,-,84665,88,9606,9606
2,entrez gene/locuslink:90,entrez gene/locuslink:2339,biogrid:106605|entrez gene/locuslink:ACVR1,biogrid:108625|entrez gene/locuslink:FNTA,entrez gene/locuslink:ACTRI(gene name synonym)...,entrez gene/locuslink:FPTA(gene name synonym)|...,"psi-mi:""MI:0018""(two hybrid)",Wang T (1996),pubmed:8599089,taxid:9606,taxid:9606,"psi-mi:""MI:0407""(direct interaction)","psi-mi:""MI:0463""(biogrid)",biogrid:183,-,90,2339,9606,9606
3,entrez gene/locuslink:2624,entrez gene/locuslink:5371,biogrid:108894|entrez gene/locuslink:GATA2,biogrid:111384|entrez gene/locuslink:PML,entrez gene/locuslink:DCML(gene name synonym)|...,entrez gene/locuslink:MYL(gene name synonym)|e...,"psi-mi:""MI:0018""(two hybrid)",Tsuzuki S (2000),pubmed:10938104,taxid:9606,taxid:9606,"psi-mi:""MI:0407""(direct interaction)","psi-mi:""MI:0463""(biogrid)",biogrid:278,-,2624,5371,9606,9606
4,entrez gene/locuslink:6118,entrez gene/locuslink:6774,biogrid:112038|entrez gene/locuslink:RPA2|entr...,biogrid:112651|entrez gene/locuslink:STAT3,entrez gene/locuslink:REPA2(gene name synonym)...,entrez gene/locuslink:ADMIO(gene name synonym)...,"psi-mi:""MI:0018""(two hybrid)",Kim J (2000),pubmed:10875894,taxid:9606,taxid:9606,"psi-mi:""MI:0407""(direct interaction)","psi-mi:""MI:0463""(biogrid)",biogrid:418,-,6118,6774,9606,9606
5,entrez gene/locuslink:375,entrez gene/locuslink:23163,biogrid:106870|entrez gene/locuslink:ARF1,biogrid:116775|entrez gene/locuslink:GGA3,-,-,"psi-mi:""MI:0018""(two hybrid)",DellAngelica EC (2000),pubmed:10747089,taxid:9606,taxid:9606,"psi-mi:""MI:0407""(direct interaction)","psi-mi:""MI:0463""(biogrid)",biogrid:586,-,375,23163,9606,9606
6,entrez gene/locuslink:377,entrez gene/locuslink:23647,biogrid:106872|entrez gene/locuslink:ARF3,biogrid:117174|entrez gene/locuslink:ARFIP2,-,entrez gene/locuslink:POR1(gene name synonym),"psi-mi:""MI:0018""(two hybrid)",Kanoh H (1997),pubmed:9038142,taxid:9606,taxid:9606,"psi-mi:""MI:0407""(direct interaction)","psi-mi:""MI:0463""(biogrid)",biogrid:612,-,377,23647,9606,9606
7,entrez gene/locuslink:377,entrez gene/locuslink:27236,biogrid:106872|entrez gene/locuslink:ARF3,biogrid:118084|entrez gene/locuslink:ARFIP1,-,entrez gene/locuslink:HSU52521(gene name synonym),"psi-mi:""MI:0018""(two hybrid)",Kanoh H (1997),pubmed:9038142,taxid:9606,taxid:9606,"psi-mi:""MI:0407""(direct interaction)","psi-mi:""MI:0463""(biogrid)",biogrid:617,-,377,27236,9606,9606
8,entrez gene/locuslink:54464,entrez gene/locuslink:226,biogrid:119970|entrez gene/locuslink:XRN1,biogrid:106728|entrez gene/locuslink:ALDOA,entrez gene/locuslink:SEP1(gene name synonym),entrez gene/locuslink:ALDA(gene name synonym)|...,"psi-mi:""MI:0018""(two hybrid)",Lehner B (2004),pubmed:15231747,taxid:9606,taxid:9606,"psi-mi:""MI:0407""(direct interaction)","psi-mi:""MI:0463""(biogrid)",bio

[6416,
 84665,
 90,
 2624,
 6118,
 375,
 377,
 377,
 54464,
 351,
 333,
 10370,
 2033,
 338,
 409,
 1436,
 7916,
 27257,
 6521,
 602,
 153,
 672,
 672,
 672,
 672,
 421,
 5092,
 5664,
 825,
 3708,
 9223,
 5925,
 7251,
 4998,
 5000,
 4174,
 8317,
 4999,
 6118,
 10926,
 10270,
 4171,
 6117,
 6119,
 5001,
 4172,
 7534,
 81704,
 9322,
 1033,
 5932,
 9464,
 1213,
 3486,
 7295,
 1386,
 1386,
 896,
 10928,
 5479,
 1465,
 1445,
 1445,
 1634,
 6493,
 6492,
 266743,
 1839,
 5111,
 11073,
 2316,
 27258,
 25804,
 23658,
 2033,
 1387,
 4926,
 54512,
 5335,
 3895,
 3791,
 133522,
 6597,
 79039,
 2099,
 2260,
 7040,
 7341,
 51678,
 7916,
 2316,
 2260,
 2566,
 2810,
 7041,
 2908,
 2645,
 3309,
 6514,
 6517,
 2898,
 2898,
 2897,
 2897,
 904,
 10287,
 10287,
 10287,
 10287,
 3298,
 3984,
 2252,
 7916,
 7916,
 7916,
 708,
 3646,
 7916,
 3178,
 7916,
 4670,
 7916,
 7916,
 7916,
 9898,
 10989,
 7916,
 7916,
 7916,
 8100,
 54464,
 3064,
 9001,
 3064,
 25764,
 9322,
 4209,
 6929,
 6929,
 3480,
 7531,
 8651,
